In [1]:
import re
import spacy
import logging
from emoji import demojize
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from langdetect import detect

Инциализация 

In [3]:
# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def initialize_spacy_models():
    """
    Инициализирует SpaCy модели для каждого языка.
    """
    return {
        "en": spacy.load("en_core_web_sm"),
        "ru": spacy.load("ru_core_news_sm"),
        "lt": spacy.load("lt_core_news_sm")
    }

def lt_model_init():
    """
    innit lithuanian emotion classifier
    :return: pipeline
    """
    tokenizer = AutoTokenizer.from_pretrained(r'C:\Users\Viktor\Documents\TMS\NLP\NLP_psyhology_support\models\lt_emotion_model_v1.1')
    model = AutoModelForSequenceClassification.from_pretrained(r'C:\Users\Viktor\Documents\TMS\NLP\NLP_psyhology_support\models\lt_emotion_model_v1.1')
    emotion_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)
    return emotion_classifier

def initialize_emotion_classifiers():
    """
    Инициализирует классификаторы эмоций для каждого языка.
    """
    return {
        "ru": pipeline("text-classification", model="seara/rubert-tiny2-russian-emotion-detection-ru-go-emotions", top_k=None),
        "en": pipeline("text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None),
        "lt": lt_model_init()
    }

spacy_models = initialize_spacy_models()
emotion_classifiers = initialize_emotion_classifiers()


2025-01-22 13:59:58,740 - INFO - Loading dictionaries from C:\Users\Viktor\Documents\TMS\NLP\NLP_psyhology_support\venv\Lib\site-packages\pymorphy3_dicts_ru\data
2025-01-22 13:59:58,774 - INFO - format: 2.4, revision: 417150, updated: 2022-01-08T22:09:24.565962
Device set to use cpu
Device set to use cpu
Device set to use cpu


Прердварительная обработка текста.

In [4]:
def detect_language(text):
    """
    Определяет язык текста.
    :param text: Входной текст
    :return: Код языка (ru, en, lt)
    """
    return detect(text)

In [7]:
def preprocess_text(text):
    """
    Обработка текста: приведение к нижнему регистру, очистка, токенизация, лемматизация, удаление стоп-слов.
    :param text: входной текст
    :param lang: язык текста (ru, en, lt)
    :return: список лемматизированных токенов
    """
    # Приведение текста к нижнему регистру
    text = text.lower()

    # Удаление HTML-тегов
    text = re.sub(r"<.*?>", "", text)

    # Удаление эмодзи
    text = demojize(text)

    # Удаление лишних символов
    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    
    lang = detect_language(text)
    # Токенизация и лемматизация
    tokens = []
    if lang in ["ru", "en", "lt"]:
        doc = spacy_models[lang](text)
        tokens = [
            token.lemma_ 
            for token in doc 
            if token.text.lower() not in spacy_models[lang].Defaults.stop_words and not token.is_punct
        ]
        logging.info(f"Лемматизация для языка {lang}: {tokens}")

    return tokens

In [5]:
def classify_emotion(text):
    """
    Классификация эмоций в тексте.
    :param text: входной текст
    :return: список эмоций с вероятностями
    """
    lang = detect_language(text)
    if lang not in emotion_classifiers:
        return lang, {"error": f"{lang} - Language not supported"}

    classifier = emotion_classifiers[lang]
    try:
        results = classifier(text)[0]
        if not results or not isinstance(results, list):
            logging.warning(f"Unexpected classifier output for language {lang}: {results}")
            return {"error": "Classifier returned unexpected results"}
        return {result['label']: result['score'] for result in results}
    except Exception as e:
        logging.error(f"Error during emotion classification for language {lang}: {e}")
        return {"error": "Failed to classify emotions"}

Тест для обработки текста

In [8]:
if __name__ == "__main__":
    texts = [
    "Я так устал после работы! Нужно отдохнуть.",
    "I am so tired after work! I need a break.",
    "Esu pavargęs po darbo. Reikia poilsio."
]

    for text in texts:
        lang = detect_language(text)
        print(f"Язык: {lang}")
        print(f"Исходный текст: {text}")
        print(f"Обработанный текст: {preprocess_text(text)}")
        print("-")
        
        
    for text in texts:
        print(f"Язык: {detect_language(text)}")
        print(f"Исходный текст: {text}")
        emotions = classify_emotion(text)
        print(f"Эмоции: {emotions}")

2025-01-22 14:01:02,257 - INFO - Лемматизация для языка ru: ['устать', 'работа', 'отдохнуть']
2025-01-22 14:01:02,281 - INFO - Лемматизация для языка en: ['tired', 'work', 'need', 'break']
2025-01-22 14:01:02,298 - INFO - Лемматизация для языка lt: ['būti', 'pavargti', 'darbas', 'reikėti', 'poilsys']


Язык: ru
Исходный текст: Я так устал после работы! Нужно отдохнуть.
Обработанный текст: ['устать', 'работа', 'отдохнуть']
-
Язык: en
Исходный текст: I am so tired after work! I need a break.
Обработанный текст: ['tired', 'work', 'need', 'break']
-
Язык: lt
Исходный текст: Esu pavargęs po darbo. Reikia poilsio.
Обработанный текст: ['būti', 'pavargti', 'darbas', 'reikėti', 'poilsys']
-
Язык: ru
Исходный текст: Я так устал после работы! Нужно отдохнуть.
Эмоции: {'sadness': 0.4984441101551056, 'caring': 0.19223575294017792, 'disappointment': 0.13804705440998077, 'neutral': 0.13139426708221436, 'joy': 0.08005299419164658, 'remorse': 0.0411076694726944, 'approval': 0.03306541219353676, 'annoyance': 0.029262900352478027, 'optimism': 0.026494909077882767, 'realization': 0.02351151593029499, 'nervousness': 0.016141101717948914, 'desire': 0.016081485897302628, 'relief': 0.015784161165356636, 'disapproval': 0.012920849956572056, 'fear': 0.01154495682567358, 'excitement': 0.010527616366744041, 'em

Модели для классификации эмоций